<a href="https://colab.research.google.com/github/newstage75/ML_study/blob/master/%E6%9C%80%E7%9F%ADPytorch/%EF%BC%98%E7%AB%A0MNIST%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E6%95%B0%E5%AD%97%E8%AA%8D%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#８章MNISTを使った数字認識

In [35]:
# 必要ライブラリの導入

!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

In [36]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from IPython.display import display

In [37]:
# torch関連ライブラリのインポート

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot

In [38]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

###GPU利用

####GPU利用のルール

1.テンソル変数はデータがCPU/GPU上のどちらにあるのかを属性として持っている

2.CPUとGPU間のデータはto関数で転送する

3.2つの変数が両方ともGPU上にある場合、演算はGPUで行われる

4.変数の片方がCPU、もう一方がGPUの場合はエラーになる。

In [39]:
#デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [40]:
#テスト用tensor変数x, y
x_np = np.arange(-2.0, 2.1, 0.25)
y_np = np.arange(-1.0, 3.1, 0.25)
x = torch.tensor(x_np).float()
y = torch.tensor(y_np).float()

z = x * y
print(z)

tensor([ 2.0000,  1.3125,  0.7500,  0.3125, -0.0000, -0.1875, -0.2500, -0.1875,
         0.0000,  0.3125,  0.7500,  1.3125,  2.0000,  2.8125,  3.7500,  4.8125,
         6.0000])


In [41]:
#変数xをGPUに送る
x = x.to(device)

print('x: ', x.device)
print('y: ', y.device)

x:  cuda:0
y:  cpu


In [42]:
#この状態の計算はエラー
z = x * y

RuntimeError: ignored

In [ ]:
#変数yもGPUに送る
y = y.to(device)

z = x * y
print(z)

In [ ]:
#データ準備1(データセットによる読み込み)

#ライブラリインポート
import torchvision.datasets as datasets

# ダウンロード先ディレクトリ名
data_root = './data'

train_set0 = datasets.MNIST(
    #元データダウンロード先の指定
    root = data_root,
    #訓練データか検証データか
    train = True,
    #元データがない場合にダウンロードするか
    download = True)

In [ ]:
#ダウンロードしたの確認

!ls -lR ./data/MNIST

In [ ]:
#データの件数の確認
print('データ件数: ', len(train_set0))

#最初の要素の取得
image, label = train_set0[0]

print('入力データの型:', type(image))
print('正解データの型:', type(label))

In [ ]:
#入力データの画像表示
plt.figure(figsize=(2,3))
plt.title(f'{label}')
plt.imshow(image,cmap='gray_r')
plt.axis('off')
plt.show()

In [ ]:
#正解データ付きで、最初の20個のイメージ表示
plt.figure(figsize=(10, 3))
for i in range(20):
  ax = plt.subplot(2, 10, i + 1)

  #imageとlabelの取得
  image, label = train_set0[i]

  #イメージ表示
  plt.imshow(image, cmap='gray_r')
  ax.set_title(f'{label}')
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
#データ前処理
#Step1 ToTensorの利用

import torchvision.transforms as transforms

transform1 = transforms.Compose([
        #　データのTensor化
        transforms.ToTensor(),
])

train_set1 = datasets.MNIST(
    root=data_root, train=True, download=True,
    transform = transform1)

In [ ]:
#変換結果の確認
image, label = train_set1[0]
print('入力データの型: ', type(image))
print('入力データのshape: ', image.shape)
print('最小値: ', image.data.min())
print('最大値: ', image.data.max())

In [ ]:
#Step2 Notmalizeの利用
transform2 = transforms.Compose([
        #データのTensor化
        transforms.ToTensor(),

        #データの正規化
        transforms.Normalize(0.5, 0.5),
])

train_set2 = datasets.MNIST(
    root = data_root, train = True, download = True,
    transform = transform2)

In [ ]:
#変換結果の確認
image, label = train_set2[0]
print('shape: ', image.shape)
print('最小値: ', image.data.min())
print('最大値: ', image.data.max())

In [ ]:
#Step3 Lambdaを利用して1階テンソル化
transform3 = transforms.Compose([
        # データのTensor化
        transforms.ToTensor(),

        #データの正規化
        transforms.Normalize(0.5, 0.5),

        #Tensorの1階テンソル化
        transforms.Lambda(lambda x: x.view(-1)),
])

train_set3 = datasets.MNIST(
    root = data_root, train = True,
    download=True, transform=transform3)

In [ ]:
#変換結果の確認
image, label = train_set3[0]
print('shape: ', image.shape)
print('最小値: ', image.data.min())
print('最大値: ', image.data.max())

## 最終的な実装

#### データ変換用関数 Transforms
#### (1) Imageをテンソル化
#### (2) [0, 1]の範囲の値を[-1, 1]の範囲にする
#### (3) データのshapeを[1, 28, 28]から[784]に変換


In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0.5, 0.5),
        transforms.Lambda(lambda x: x.view(-1)),
])

In [ ]:
#データ取得用関数Dataset

#訓練用データセットの定義
train_set = datasets.MNIST(
    root = data_root, train = True,
    download = True, transform = transform)

#検証データセットの定義
test_set = datasets.MNIST(
    root = data_root, train = False,
    download = True, transform = transform)

### データローダーによるミニバッチ用データ生成

In [ ]:
from torch.utils.data import DataLoader

batch_size = 500

#訓練用データローダー（シャッフルもする）
train_loader = DataLoader(
    train_set, batch_size = batch_size,
    shuffle = True)

#検証用データローダー(シャッフル不要)
test_loader = DataLoader(
    train_set, batch_size = batch_size,
    shuffle = False)